# Phần preprocessing

Thư viện

In [40]:
import os
import numpy as np
import pandas as pd

Định nghĩa các thông số

In [41]:
step = 25 # Time_step
X = [] # Data sau cùng
Y = [] # Label (1 là tục, 0 là không tục)

Lấy data

In [42]:
list_folder_name = os.listdir('DATA')
for folder in list_folder_name:
    path_folder_parent = os.path.join('DATA', folder)
    
    path_folder_tuc = os.path.join(path_folder_parent, 'DATA_TUC')
    path_folder_khong_tuc = os.path.join(path_folder_parent, 'DATA_KHONG_TUC')

    # Data tục
    for file in os.listdir(path_folder_tuc):
        if file.endswith('.csv'):
            path_file_csv = os.path.join(path_folder_tuc, file)
            data = pd.read_csv(path_file_csv).to_numpy()
            for i in range(step, data.shape[0]):
                X.append(data[i - step: i, :])
                Y.append(1)
                
    # Data không tục
    for file in os.listdir(path_folder_khong_tuc):
        if file.endswith('.csv'):
            path_file_csv = os.path.join(path_folder_khong_tuc, file)
            data = pd.read_csv(path_file_csv).to_numpy()
            
            for i in range(step, data.shape[0]):
                X.append(data[i - step: i, :])
                Y.append(0)

Chuyển về numpy

In [43]:
X = np.array(X)
Y = np.array(Y)
Y = Y.reshape(-1, 1)
print(X.shape, Y.shape)

(7777, 25, 80) (7777, 1)


# Phần này là training bằng tensorflow, muốn training bằng pytorch thì chuyển numpy sang tensor và training    

Chia dữ liệu

In [44]:
import pickle
pickle.dump(X, open('X.pkl', 'wb'))
pickle.dump(Y, open('Y.pkl', 'wb'))

In [45]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2)


Tạo model

In [46]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Bidirectional, GlobalAveragePooling1D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# Tham số mô hình
TIME_STEPS = step  # Số bước thời gian (chuỗi đầu vào)
FEATURES = 80    # Số đặc trưng mỗi bước thời gian

# Input Layer
inputs = Input(shape=(TIME_STEPS, FEATURES))

# LSTM Layer 1
x = Bidirectional(LSTM(64, activation='tanh', return_sequences=True, kernel_regularizer=l2(1e-4)))(inputs)

# LSTM Layer 2
x = Bidirectional(LSTM(128, activation='tanh', return_sequences=True, kernel_regularizer=l2(1e-4)))(x)
x = BatchNormalization()(x)
x = Dropout(0.15)(x)

# LSTM Layer 3
x = Bidirectional(LSTM(256, activation='tanh', return_sequences=True, kernel_regularizer=l2(1e-4)))(x)
x = BatchNormalization()(x)
x = Dropout(0.15)(x)

# LSTM Layer 3
x = Bidirectional(LSTM(512, activation='tanh', return_sequences=True, kernel_regularizer=l2(1e-4)))(x)
x = BatchNormalization()(x)
x = Dropout(0.15)(x)

# GlobalAveragePooling1D để giảm chiều dữ liệu
x = GlobalAveragePooling1D()(x)
x = Dropout(0.2)(x)

# Dense Layer
x = Dense(16, activation="tanh", kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.2)(x)

# Output Layer (Binary Classification)
outputs = Dense(1, activation="sigmoid")(x)

# Tạo model
model = Model(inputs, outputs)

# Giảm Learning Rate (1e-4 hoặc 1e-5 nếu cần)
optimizer = Adam(learning_rate=1e-5)

# Compile model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall'])

# In tóm tắt mô hình
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 25, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 25, 128)        │        74,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 25, 256)        │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 25, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 25, 512)        │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 25, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 25, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 25, 1024)       │     4,198,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 25, 1024)       │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 25, 1024)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │        16,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,610,017 (21.40 MB)

 Trainable params: 5,606,433 (21.39 MB)

 Non-trainable params: 3,584 (14.00 KB)

Training

In [47]:
history = model.fit(xtrain, ytrain, batch_size= 16, epochs= 20, validation_data= (xtest, ytest))

Epoch 1/20
 65/389 ━━━━━━━━━━━━━━━━━━━━ 1:27 269ms/step - accuracy: 0.5175 - loss: 1.0741 - precision: 0.6444 - recall: 0.5400

KeyboardInterrupt: 

Lưu model

In [ ]:
model.save('model_lstm.h5')